In [59]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
import pandas as pd
import yaml
import geopandas as gp
import time
import pandas as pd
from geopy.distance import great_circle
import folium
import h5py

In [60]:
import sys
sys.path.append(r'../tools_F-J/')
from toollib_standard import maplib
from toollib_standard import mathlib
from toollib_standard import filelib
from toollib_standard import plotlib

In [61]:
with open('a-project_repar.yml', 'r', encoding='utf-8') as f:
    proj = yaml.load(f.read(), Loader=yaml.FullLoader)
name_project = proj['name']
#name_project = 'project_repartrition/output_repar_01-03/'               # Harmon server
name_project

'project_repartition_v4.0/output_repar_v9.5_01--10-16Hz/'

In [62]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

dir_CC_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project:  /shdisk/rem2/Harmon/F-J/San/project_repartition_v4.0/output_repar_v9.5_01--10-16Hz/


In [63]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [64]:
dir_CC = dir_CC_workspace + info_basic['rdir_CC']
dir_ds = dir_project + info_basic['rdir_ds']
dir_partition = dir_project + info_basic['rdir_partition']
dir_image = dir_project + info_basic['rdir_image'] + 'regular_FJ/'
if os.path.exists(dir_image) == False:
    os.makedirs(dir_image)

In [65]:
key_subworks = info_basic['key_subworks']
key_subworks

['1--08-03',
 '2--09-03',
 '3--10-03',
 '4--11-03',
 '5--12-03',
 '6--13-03',
 '7--14-03',
 '8--15-03',
 '9--16-03',
 '10--18-03',
 '12--20-03',
 '13--21-03',
 '15--23-03',
 '17--25-03',
 '18--26-03',
 '19--27-03',
 '20--28-03',
 '21--29-03',
 '22--30-03',
 '23--31-03',
 '25--33-03',
 '26--34-03',
 '28--36-03',
 '30--41-03',
 '31--42-03',
 '33--46-03',
 '34--47-03',
 '35--48-03',
 '36--52-03',
 '37--53-03',
 '38--07-04',
 '39--08-04',
 '40--09-04',
 '41--10-04',
 '42--12-04',
 '43--13-04',
 '44--17-04',
 '45--20-04',
 '46--21-04',
 '48--23-04',
 '49--24-04',
 '50--25-04',
 '51--27-04',
 '52--28-04',
 '53--29-04',
 '54--30-04',
 '55--31-04',
 '56--32-04',
 '57--33-04',
 '58--34-04',
 '59--35-04',
 '60--36-04',
 '63--47-04',
 '65--49-04',
 '66--52-04',
 '67--53-04',
 '68--08-05',
 '69--12-05',
 '70--13-05',
 '71--14-05',
 '72--16-05',
 '75--22-05',
 '76--24-05',
 '77--26-05',
 '78--27-05',
 '79--28-05',
 '80--29-05',
 '81--30-05',
 '84--35-05',
 '85--36-05',
 '86--37-05',
 '88--39-05',
 

In [66]:
stalistname_all = info_basic['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
nsta_all = len(stainfo.iloc[:,0])
StationPairs_all = mathlib.GetStationPairs(nsta_all)
nPairs_all = int(len(StationPairs_all)/2)
stalist_all = stainfo['Station'].tolist()
lat_stations_all = stainfo['latitude'].tolist() 
lon_stations_all = stainfo['longitude'].tolist()

In [67]:
stalistname_all = info_basic['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
nsta_all = len(stainfo.iloc[:,0])
StationPairs_all = mathlib.GetStationPairs(nsta_all)
nPairs_all = int(len(StationPairs_all)/2)
stalist_all = stainfo['Station'].tolist()
lat_stations_all = stainfo['latitude'].tolist() 
lon_stations_all = stainfo['longitude'].tolist()

In [68]:
stations = {}
lat = {}
lon = {}
for key in key_subworks:
    filepath = dir_partition + str(key) + '.txt'
    stations_this, lat_stations_this, lon_stations_this = np.loadtxt(filepath, dtype='str' , unpack=True)
    stations[key] = stations_this
    lat[key] = lat_stations_this.astype(float)
    lon[key] = lon_stations_this.astype(float)

In [69]:
faults = np.load('clark_faults.npy', allow_pickle='TRUE').item()

In [70]:
ncffile = h5py.File(dir_CC + 'gather_all.h5','r')
ncfs = ncffile['ncfs'][:]
f = ncffile['f'][:]
count_all = ncffile['count'][:]
ncffile.close()

In [71]:
def Pairs(sta):
    p = []
    nsta = len(sta)
    for ii in range(nsta):
        for jj in range(ii+1,nsta):
            p.append([sta[ii],sta[jj]])
    return p
def cal_indx(pair,nsta):
    indx = int(pair[0]*(2*nsta-pair[0]-1)/2+pair[1]-pair[0]-1)
    return indx

In [72]:
def time_window_filter(t,ncfst0,r,v_min,t0,a):
    ncfst = ncfst0.copy()
    for i in range(len(ncfst)):
        tag = r[i]/v_min
        #print(t0,tag)
        t1 = t[t>-tag-t0][t[t>-tag-t0]< tag+t0]
        start = np.where(t == t1[0])[0][0]
        end = np.where(t == t1[-1])[0][0]
        ncfst[i][start:end+1]= ncfst[i][start:end+1]* np.exp(-a*np.abs((tag-np.abs(t1))))
    return ncfst

In [73]:
def remove_stack(key_subwork,ncfs_sum_linear,r,StationPairs):
    global dir_project
    global key_subworks
    global info_basic
    global flag_save
    global v_tag
    global t0
    global a

    start0 = time.time()
    f0 = info_basic_bi['f']
    dt = 1/np.max(f0)
    t = (np.linspace(-len(f0)-1,len(f0)-1,2*(len(f0)-1))+0.5)*dt/2
    ncfst_linear = mathlib.freq_time(ncfs_sum_linear)
    ncfst1 = time_window_filter(t,ncfst_linear,r,v_tag,t0,a)
    ncfs_sum_remove = mathlib.time_freq(ncfst1)
    if flag_save:
        tag = name_project[ name_project.rfind('_')+1: -1]
        dir_stack = dir_project + 'stack_'+tag+'/'
        outname = key_subwork+'_gather_timewindow.h5'
        if os.path.exists(dir_stack+outname):
            os.remove(dir_stack+outname)
        ncffile = h5py.File(dir_stack+outname,'w')
        ncffile.create_dataset('ncfs', data=ncfs_sum_remove) 
        ncffile.create_dataset('r',data=r)
        ncffile.create_dataset('StationPairs',data=StationPairs)
        ncffile.close()
    print('Finish remove stack, time:', time.time()-start0, ' seconds')
    return ncfs_sum_remove

In [74]:
v_tag = info_basic['remove_v_tag']
t0 = info_basic['remove_t0']
a = info_basic['remove_a']

In [75]:
flag_save = 0

start = 0 
interval = 10
flag_time = 0.02
tao = 0

c_map = 'jet'
v_min = 0.1
v_max = None
d_len = 31

xlim_f = [2,30]
xlim_T = [-1,1]

info_basic['v_min'] = v_min
info_basic['v_max'] = v_max
info_basic['c_map'] = c_map

In [76]:
ds_linear_all = {}
ds_remove_all = {}
index_ncfs = {}
rs = {}
for key_subwork in key_subworks:
    print(key_subwork+ '; '+str(key_subworks.index(key_subwork)+1)+'/'+str(len(key_subworks))+' subworks.')
    ds = h5py.File(dir_ds+'ds_'+str(key_subwork)+'.h5', 'r')
    ds_linear = plotlib.smooth_ds(ds['ds_linear'][0])
    ds_remove = plotlib.smooth_ds(ds['ds_remove'][0])
    ds_linear_all[key_subwork] = ds_linear
    ds_remove_all[key_subwork] = ds_remove
    index_ncfs[key_subwork] = list(ds['index_ncfs'])
    rs[key_subwork] = np.array(ds['r'])
    ds.close()

1--08-03; 1/466 subworks.
2--09-03; 2/466 subworks.
3--10-03; 3/466 subworks.
4--11-03; 4/466 subworks.
5--12-03; 5/466 subworks.
6--13-03; 6/466 subworks.
7--14-03; 7/466 subworks.
8--15-03; 8/466 subworks.
9--16-03; 9/466 subworks.
10--18-03; 10/466 subworks.
12--20-03; 11/466 subworks.
13--21-03; 12/466 subworks.
15--23-03; 13/466 subworks.
17--25-03; 14/466 subworks.
18--26-03; 15/466 subworks.
19--27-03; 16/466 subworks.
20--28-03; 17/466 subworks.
21--29-03; 18/466 subworks.
22--30-03; 19/466 subworks.
23--31-03; 20/466 subworks.
25--33-03; 21/466 subworks.
26--34-03; 22/466 subworks.
28--36-03; 23/466 subworks.
30--41-03; 24/466 subworks.
31--42-03; 25/466 subworks.
33--46-03; 26/466 subworks.
34--47-03; 27/466 subworks.
35--48-03; 28/466 subworks.
36--52-03; 29/466 subworks.
37--53-03; 30/466 subworks.
38--07-04; 31/466 subworks.
39--08-04; 32/466 subworks.
40--09-04; 33/466 subworks.
41--10-04; 34/466 subworks.
42--12-04; 35/466 subworks.
43--13-04; 36/466 subworks.
44--17-04;

In [77]:
#%%capture
for key_subwork in key_subworks:
    print(key_subwork+ '; '+str(key_subworks.index(key_subwork)+1)+'/'+str(len(key_subworks))+' subworks.')
    #ncfs_sum_linear, r, StationPairs = linear_stack(key_subwork)
    
    index = index_ncfs[key_subwork]
    ncfs_sum_linear = np.array(ncfs)[index,:]
    r = rs[key_subwork]

    stalist = stations[key_subwork]
    lat_this = lat[key_subwork]
    lon_this = lon[key_subwork]
    nsta = len(stalist)
    StationPairs = mathlib.GetStationPairs(nsta)
    nPairs = int(len(StationPairs)/2)

    ncfs_sum_remove = remove_stack(key_subwork,ncfs_sum_linear,r,StationPairs)

    fig,ax = plt.subplots(3,3,figsize= (22,18))
    # plot area
    ax[0][0]=plotlib.plot_area(ax[0][0],lon_stations_all,lat_stations_all,lon_this,lat_this)
    for i in range(len(faults)):
        ax[0][0].plot(faults['clark'+str(i+1)]['lon'], faults['clark'+str(i+1)]['lat'], 'k')

    f0 = info_basic_bi['f']
    t = info_basic_bi['t']
    ncfst_linear = mathlib.freq_time(ncfs_sum_linear)
    ncfst_remove = mathlib.freq_time(ncfs_sum_remove)
    
    title1 = "Linear Stack subarea frequency for "+str(key_subwork)
    #xlim_f = [0,15]
    ax[0][1] = plotlib.plot_ncfs(ax[0][1],f0,ncfs_sum_linear,r,title1,xlim_f,0)
    ax[0][2] = plotlib.plot_ncfs(ax[0][2],f0,ncfs_sum_remove,r,title1,xlim_f,0)
    
    # plot fj
    c = np.linspace(info_basic['fj_c_min'],info_basic['fj_c_max'],info_basic['fj_c_num'])
    
    title0 = "Linear Stack subarea time for "+str(key_subwork)
    ax[1][0] = plotlib.plot_ncfst(ax[1][0],t,ncfst_linear[start::interval],r[start::interval],title0,flag_time,xlim_T,0)
    title0 = "Remove Stack subarea time for "+str(key_subwork)
    ax[1][1] = plotlib.plot_ncfst(ax[1][1],t,ncfst_remove[start::interval],r[start::interval],title0,flag_time,xlim_T,0)
    for i in range(len(r)):
        ax[1][1].plot()
    
    title0 = "Linear stack dispersion curve "+str(d_len)+' days'
    ax[2][0] = plotlib.plot_fj(ax[2][0],ds_linear,title0,f0,c,0,v_min=v_min,v_max=v_max,c_map=c_map)
    ax[2][0].set_xlim(xlim_f)

    title0 = "Remove stack dispersion curve "+str(d_len)+' days'
    ax[2][1] = plotlib.plot_fj(ax[2][1],ds_remove,title0,f0,c,0,v_min=v_min,v_max=v_max,c_map=c_map)
    ax[2][1].set_xlim(xlim_f)

    plt.tight_layout()
    plt.savefig(dir_image+str(key_subwork)+'_compare_remove.png',dpi=100)
    plt.close()

1--08-03; 1/466 subworks.
Finish remove stack, time: 0.11320877075195312  seconds
2--09-03; 2/466 subworks.
Finish remove stack, time: 0.10875368118286133  seconds
3--10-03; 3/466 subworks.
Finish remove stack, time: 0.14082121849060059  seconds
4--11-03; 4/466 subworks.
Finish remove stack, time: 0.16145658493041992  seconds
5--12-03; 5/466 subworks.
Finish remove stack, time: 0.18662500381469727  seconds
6--13-03; 6/466 subworks.
Finish remove stack, time: 0.1615581512451172  seconds
7--14-03; 7/466 subworks.
Finish remove stack, time: 0.14613652229309082  seconds
8--15-03; 8/466 subworks.
Finish remove stack, time: 0.14395976066589355  seconds
9--16-03; 9/466 subworks.
Finish remove stack, time: 0.1604290008544922  seconds
10--18-03; 10/466 subworks.
Finish remove stack, time: 0.20815062522888184  seconds
12--20-03; 11/466 subworks.
Finish remove stack, time: 0.32694220542907715  seconds
13--21-03; 12/466 subworks.
Finish remove stack, time: 0.13774371147155762  seconds
15--23-03; 1

In [78]:
with open(dir_project+'Basic_info.yml', 'w', encoding='utf-8') as f:
   yaml.dump(data=info_basic, stream=f, allow_unicode=True)